In [1]:
%load_ext autoreload
%autoreload 2

from hashlib import sha256

In [19]:
p.match('http://asdfkj/').group(1)

'asdfkj'

# Blockchain

In [24]:
from blockchain import Blockchain

import re

p = re.compile('http://([^/]+)')

class MockRequests(object):
    def __init__(self, blockchain_for_node):
        self.blockchain_for_node = blockchain_for_node
        
    def get(self, url):
        node = p.match(url).group(1)
        return self.blockchain_for_node[node].chain


## Have alice, bob, and charlotte make blockchain objects and register their nodes

In [2]:
bcs = {
    'alice': Blockchain(),
    'bob': Blockchain(),
    'charlotte': Blockchain()
}
for node1, bc1 in bcs.items():
    for node2, bc2 in bcs.items():
        if node1 != node2:
            bcs[node1].register_node(node2)

See the transactions

In [3]:
bcs

{'alice': {
   "chain_length": 1,
   "n_current_transactions": 0,
   "n_nodes": 2
 }, 'bob': {
   "chain_length": 1,
   "n_current_transactions": 0,
   "n_nodes": 2
 }, 'charlotte': {
   "chain_length": 1,
   "n_current_transactions": 0,
   "n_nodes": 2
 }}

See the chains

In [4]:
for node, bc in bcs.items():
    print("{}:\n{}\n".format(node, bc.chain))

alice:
[{'index': 1, 'timestamp': 1536179323.287318, 'transactions': [], 'proof': 100, 'previous_hash': 1}]

bob:
[{'index': 1, 'timestamp': 1536179323.287321, 'transactions': [], 'proof': 100, 'previous_hash': 1}]

charlotte:
[{'index': 1, 'timestamp': 1536179323.287323, 'transactions': [], 'proof': 100, 'previous_hash': 1}]



## Make transactions

Have alice give bob 100

In [11]:
bcs['alice'].new_transaction(sender='alice', recipient='bob', amount=100)
bcs['alice']

{
  "chain_length": 2,
  "n_current_transactions": 1,
  "n_nodes": 2
}

In [12]:
bcs['alice'].currentTransactions

[{'sender': 'alice', 'recipient': 'bob', 'amount': 100}]

Have bob give charlotte 50

In [13]:
bcs['bob'].new_transaction(sender='bob', recipient='charlotte', amount=50)
bcs['bob']

{
  "chain_length": 2,
  "n_current_transactions": 1,
  "n_nodes": 2
}

What are the transactions...

In [14]:
bcs['bob'].currentTransactions

[{'sender': 'bob', 'recipient': 'charlotte', 'amount': 50}]

## Mine

In [9]:
blocks = dict()
for node, bc in bcs.items():
    blocks[node] = bc.mine()
blocks

{'alice': {'message': 'New block forged',
  'index': 2,
  'transactions': [{'sender': 'alice', 'recipient': 'bob', 'amount': 100},
   {'sender': '0',
    'recipient': 'ff64505e6b9b41c1b704a1fedef009b1',
    'amount': 1}],
  'proof': 363380,
  'previous_hash': '7978732c9eb72e670824267bee74eb6344b1be8a4524f78826b344b3feb754e9'},
 'bob': {'message': 'New block forged',
  'index': 2,
  'transactions': [{'sender': 'bob', 'recipient': 'charlotte', 'amount': 50},
   {'sender': '0',
    'recipient': 'ff64505e6b9b41c1b704a1fedef009b1',
    'amount': 1}],
  'proof': 85008,
  'previous_hash': '0eddde155fac8cc863aaa2cd9a0d67a79c077906e0a334b3c0016d69fcd5ed40'},
 'charlotte': {'message': 'New block forged',
  'index': 2,
  'transactions': [{'sender': '0',
    'recipient': 'ff64505e6b9b41c1b704a1fedef009b1',
    'amount': 1}],
  'proof': 97509,
  'previous_hash': 'f46637125a1a869e2ec8b0815c18bc2ae5287aebb9b5a994afc95af14aff63f5'}}

In [10]:
bcs

{'alice': {
   "chain_length": 2,
   "n_current_transactions": 0,
   "n_nodes": 2
 }, 'bob': {
   "chain_length": 2,
   "n_current_transactions": 0,
   "n_nodes": 2
 }, 'charlotte': {
   "chain_length": 2,
   "n_current_transactions": 0,
   "n_nodes": 2
 }}

In [26]:
req = MockRequests(bcs)

In [27]:
req.get('http://alice/chain')

[{'index': 1,
  'timestamp': 1536179323.287318,
  'transactions': [],
  'proof': 100,
  'previous_hash': 1},
 {'index': 2,
  'timestamp': 1536179330.029762,
  'transactions': [{'sender': 'alice', 'recipient': 'bob', 'amount': 100},
   {'sender': '0',
    'recipient': 'ff64505e6b9b41c1b704a1fedef009b1',
    'amount': 1}],
  'proof': 363380,
  'previous_hash': '7978732c9eb72e670824267bee74eb6344b1be8a4524f78826b344b3feb754e9'}]

In [ ]:
req